## Introduction

The goal of the BeeAI project is to make AI agents interoperable, regardless of their underlying implementation. The project consists of two key components:
- **BeeAI Platform**: The platform to easily discover, run, and compose AI agents from any framework.
- **BeeAI Framework**: A production-grade framework for building AI agents in either Python or TypeScript.

Detailed information on BeeAI can be found [here](https://beeai.dev/).

### What's in this notebook?

This notebook demonstrates fundamental usage patterns of the BeeAI Framework in Python. The examples progressively increase in complexity, providing a well-rounded overview of the framework.

You can run this notebook on [**Google Colab**](https://colab.research.google.com/). The notebook uses **Ollama** to provide access to a variety of foundation models for remote execution. The notebook will run faster on Colab if you use the free *T4 GPU* option by selecting *Runtime / Change runtime type* in the Colab system menu.

Run the Next Cell to wrap Notebook output.

In [ ]:
from IPython.display import HTML, display
def set_css():
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))

get_ipython().events.register("pre_run_cell", set_css)

### Install Libraries
We start by installing the required dependencies and starting Ollama server.

In [ ]:
%pip install -q langchain_community wikipedia requests==2.32.4 beeai-framework

!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &
!ollama pull granite3.3:8b


### Import Libraries


In [ ]:
from typing import Literal
from IPython.display import Markdown
from beeai_framework.backend.message import UserMessage, AssistantMessage, SystemMessage
from beeai_framework.template import PromptTemplate, PromptTemplateInput
from beeai_framework.backend.chat import ChatModel, ChatModelOutput
from beeai_framework.memory.unconstrained_memory import UnconstrainedMemory
from pydantic import BaseModel, Field

def object_on_screen(obj):
  display(obj)

print("Imports and credentials completed")

## Prompt Templates

One of the core constructs in the BeeAI Framework is the PromptTemplate. It allows you to dynamically insert data into a prompt before sending it to a language model. BeeAI uses the Mustache templating language for prompt formatting.

The following example demonstrates how to create a Retrieval-Augmented Generation (RAG) template and apply it to your data to generate a structured prompt.

In [ ]:
print("RAG template that includes a Context and a Question:")

# Define the structure of the input data that can passed to the template i.e. the input schema
class RAGTemplateInput(BaseModel):
    question: str
    context: str


# Define the prompt template
rag_template: PromptTemplate = PromptTemplate(
    PromptTemplateInput(
        schema=RAGTemplateInput,
        template="""
Context: {{context}}
Question: {{question}}

Provide a concise answer based on the context. Avoid statements such as 'Based on the context' or 'According to the context' etc. """,
    )
)

# Render the template using an instance of the input model
prompt = rag_template.render(
    RAGTemplateInput(
        question="What is the capital of France?",
        context="France is a country in Europe. Its capital city is Paris, known for its culture and history.",
    )
)

# Print the rendered prompt
html = Markdown(prompt)   # convert to HTML
object_on_screen(html)


## More Complex Templates

The previous example demonstrated a simple template, but the PromptTemplate class can also handle more complex structures and incorporate conditional logic.

The following example showcases a template that includes a question along with a set of detailed search results represented as a list.

In [ ]:
print("Add to our PromptTemplate a website and some search results:")

# Individual search result
class SearchResult(BaseModel):
    title: str
    url: str
    content: str


# Input specification
class SearchTemplateInput(BaseModel):
    question: str
    results: list[SearchResult]


# Define the template, in this instance the template will iterate over the results
search_template: PromptTemplate = PromptTemplate(
    PromptTemplateInput(
        schema=SearchTemplateInput,
        template="""
Search results:
{{#results.0}}
{{#results}}
Title: {{title}}
Url: {{url}}
Content: {{content}}
{{/results}}
{{/results.0}}

Question: {{question}}
Provide a concise answer based on the search results provided.""",
    )
)

# Render the template using an instance of the input model
prompt = search_template.render(
    SearchTemplateInput(
        question="What is the capital of France?",
        results=[
            SearchResult(
                title="France",
                url="https://en.wikipedia.org/wiki/France",
                content="France is a country in Europe. Its capital city is Paris, known for its culture and history.",
            )
        ],
    )
)

# Print the rendered prompt
html = Markdown(prompt)   # convert to HTML
object_on_screen(html)

## The ChatModel

Once you have a PromptTemplate and can easily render prompts, you’re ready to start interacting with a model. BeeAI supports a variety of LLMs through the ChatModel interface.

In this section, we will use the IBM `Granite 3.1 8B` language model via the Ollama provider.

If you haven't set up Ollama yet, follow the [guide on running Granite 3.1 using Ollama](https://www.ibm.com/granite/docs/run/granite-on-mac/granite/) for mac, or for other platforms use the [Ollama documentation](https://ollama.com) and [IBM Granite model page](https://ollama.com/library/granite3.1-dense:8b).

Before creating a ChatModel, we need to briefly discuss Messages. The ChatModel operates using message-based interactions, allowing you to structure conversations between the user and the assistant (LLM) naturally.

In [ ]:
# Construct ChatModel
model=ChatModel.from_name("ollama:granite3.3:8b")

question1 = "Briefly explain quantum computing in simple terms with an example."
message = UserMessage(content=question1)
output: ChatModelOutput = await model.create(messages=[message])
answer1 = output.get_text_content()

print("Question: " + question1)
print("Answer: " + answer1)

In [ ]:
question2 = "Hello! Can you tell me what is the capital of France?"
message = UserMessage(content=question2)

# Create a ChatModel to interface with granite3.1-dense:8b on a local ollama
# model = ChatModel.from_name("ollama:granite3.1-dense:8b")

output: ChatModelOutput = await model.create(messages=[message])
answer2 = output.get_text_content()
print("Question: " + question2)
print()
print("Answer: " + answer2)

## Memory
The model has provided a response! We can now start to build up a `Memory`. Memory is just a convenient way of storing a set of messages that can be considered as the history of the dialog between the user and the llm.

In this next example we will construct a memory from our existing messages and add a new user message. Notice that the new message can implicitly refer to content from prior messages. Internally the `ChatModel` formats all the messages and sends them to the LLM.

In [ ]:
memory = UnconstrainedMemory()
question3 = "If you had to recommend one thing to do there, what would it be?"
await memory.add_many(
    [
        message,
        AssistantMessage(content=output.get_text_content()),
        UserMessage(content=question3),
    ]
)
output: ChatModelOutput = await model.create(messages=memory.messages)
answer3 = output.get_text_content()

print("Previous Question: " + question2)
print("Previous Answer: " + answer2)
print()
print("Next Question: " + question3)
print("Next Answer: " + answer3)

## Combining Templates and Messages

To use a PromptTemplate with the Granite ChatModel, you can render the template and then place the resulting content into a Message. This allows you to dynamically generate prompts and pass them along as part of the conversation flow.

In [ ]:
# Some context that the model will use to provide an answer. Source wikipedia: https://en.wikipedia.org/wiki/Ireland
context = """The geography of Ireland comprises relatively low-lying mountains surrounding a central plain, with several navigable rivers extending inland.
Its lush vegetation is a product of its mild but changeable climate which is free of extremes in temperature.
Much of Ireland was woodland until the end of the Middle Ages. Today, woodland makes up about 10% of the island,
compared with a European average of over 33%, with most of it being non-native conifer plantations.
The Irish climate is influenced by the Atlantic Ocean and thus very moderate, and winters are milder than expected for such a northerly area,
although summers are cooler than those in continental Europe. Rainfall and cloud cover are abundant.
"""

# Lets reuse our RAG template from earlier!
question4 = "How much of Ireland is forested?"
prompt = rag_template.render(RAGTemplateInput(question=question4, context=context))
output: ChatModelOutput = await model.create(messages=[UserMessage(content=prompt)])
answer4 = output.get_text_content()

print("RAG Template:")
html = Markdown(prompt)   # convert to HTML
object_on_screen(html)
print("Answer: " + answer4)


## Structured Outputs

Often, you'll want the LLM to produce output in a specific format. This ensures reliable interaction between the LLM and your code—such as when you need the LLM to generate input for a function or tool. To achieve this, you can use structured output.

In the example below, we will prompt Granite to generate a character using a very specific format.

In [ ]:
# The output structure definition, note the field descriptions that can help the LLM to understand the intention of the field.
class CharacterSchema(BaseModel):
    name: str = Field(description="The name of the character.")
    occupation: str = Field(description="The occupation of the character.")
    species: Literal["Human", "Insectoid", "Void-Serpent", "Synth", "Ethereal", "Liquid-Metal"] = Field(
        description="The race of the character."
    )
    back_story: str = Field(description="Brief backstory of this character.")

question5 = "Create a fantasy sci-fi character for my new game. This character will be the main protagonist, be creative."
message = UserMessage(question5)
response = await model.create_structure(schema=CharacterSchema, messages=[message])

print("Question: " + question5)
print("Structured output:")
object_on_screen(response.object)

## System Prompts

The SystemMessage is a special message type that can influence the general behavior of an LLM. By including a SystemMessage, you can provide high-level instructions that shape the LLM’s overall response style. The system message typically appears as the first message in the model’s memory.

In the example below, we add a system message that instructs the LLM to speak like a pirate!

In [ ]:
pirate_message = "You are pirate. You always respond using pirate slang."
system_message = SystemMessage(content=pirate_message)
question6 = "What is a baby hedgehog called?"
message = UserMessage(content=question6)
output: ChatModelOutput = await model.create(messages=[system_message, message])
answer6 = output.get_text_content()

print("SystemMessage: " + pirate_message)
print()
print("Question: " + question6)
print("Answer: " + answer6)
print()
print("Demo complete")

## Learn More

Detailed information on BeeAI can be found [here](https://beeai.dev/).

In this notebook, you learned the basics of the BeeAI Framework, including PromptTemplates, Messages, ChatModels, Memory, Structured Outputs, and SystemPrompts.
